In [1]:
import numpy as np
import pysynth as ps
import random
import sys
import pyaudio
import wave
import winsound
import simpleaudio as sa
from pydub import AudioSegment
import tkinter as tk
from tkinter import LEFT, RIGHT
import copy
from monkeylearn import MonkeyLearn
from random import randint
ml = MonkeyLearn('lmao')

Piano key frequencies (for equal temperament):
Key number	Scientific name	Frequency (Hz)
         1	             A0	         27.50
         2	            A#0	         29.14
         3	             B0	         30.87
         4	             C1	         32.70
         5	            C#1	         34.65
         6	             D1	         36.71
         7	            D#1	         38.89
         8	             E1	         41.20
         9	             F1	         43.65
        10	            F#1	         46.25
        11	             G1	         49.00
        12	            G#1	         51.91
        13	             A1	         55.00
        14	            A#1	         58.27
        15	             B1	         61.74
        16	             C2	         65.41
        17	            C#2	         69.30
        18	             D2	         73.42
        19	            D#2	         77.78
        20	             E2	         82.41
        21	             F2	         87.31
        22	            F#2	  

C:\Users\shihyuch\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


        73	             A6	       1760.00
        74	            A#6	       1864.66
        75	             B6	       1975.53
        76	             C7	       2093.00
        77	            C#7	       2217.46
        78	             D7	       2349.32
        79	            D#7	       2489.02
        80	             E7	       2637.02
        81	             F7	       2793.83
        82	            F#7	       2959.96
        83	             G7	       3135.96
        84	            G#7	       3322.44
        85	             A7	       3520.00
        86	            A#7	       3729.31
        87	             B7	       3951.07
        88	             C8	       4186.01


In [ ]:


# high level algorithm knowledge and probabilities were taken from research from
# a Cornell research paper. I didn't use the code they had in their Github repo,
# mainly because they used multiple files, I wasn't sure how that works. I just
# used their high level logic and probability to implement my own code.

# Computoser - rule-based, probability-driven algorithmic music composition
# Link: https://www.academia.edu/9696759/Computoser_-_rule-based_probability-driven_algorithmic_music_composition

#Global Constants:
MinsPerSec = 1/60
Width = 1000
Height = 1000
totalLength = 20
key_id = 'cl_ETmJQsVx'
mood_id = 'cl_7a2qVQAb'
# Musical Constants:
step = 1
Tonic = 0
SubTonic = -1

blueDominant = 4
classicalDominant = 4
countryDominant = 5
discoDominant = 4
hiphopDominant = 5
jazzDominant = 4
metalDominant = 4
popDominant = 5
reggaeDominant = 4
rockDominant = 4



Subdominant = 3
MelodyStartOctave = 4
MSO = MelodyStartOctave
HarmonyStartOctave = 3
HSO = HarmonyStartOctave
tempo = 100


# the two individual music lines that are separately generated and merged later
finalPieceMelody = []
finalPieceHarmony = []

keyList = ["C","G","D","A","E","B","F#","C#","F","Bb","Eb","Ab","Db","Gb","Cb"] 
# list of all of the keys
noteList = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"] # starts on "C"
#            0   1    2   3    4   5   6    7   8    9   10   11
# to be transformed later to match key
# list of all of the notes


def generateMusic(key,genre,mood): # totalLength: 15-30
    global finalPieceMelody
    global finalPieceHarmony
    global noteList
    finalPieceMelody = []
    finalPieceHarmony = []
    unisonLocations = []
    
    def expandProbability(probabilityTable):
        result = []
        for list in probabilityTable:
            result += ["%s" %(list[0])]*list[1]
        return result
        
        
    ##################################
    # blue Pop Song Probabilities
    ##################################
    
    # data taken directly from the journal
    
    probabilityTypeblue = [["unison",25], # playing an octave at the same time
                                         # coordinated with harmony
                          ["octave",2],  # stepping eight notes at once
                          ["step",48],   # stepping whole step
                          ["skip",25]]   # skip refers to an interval
                   
    probabilityIntervalblue = [[4,25], # perfect fifth
                              [3,2],  # perfect fourth
                              [2,48], # third
                              [5,25]] # sixth
                              
    probabilityHarmonyblue = [[4,25], # same as interval, but less dissonance
                             [3,15],
                             [2,60]]
                       
    probabilityLengthblue = [[16,10], 
                            [8,31],       
                            [4,40],      
                            [3,7], 
                            [2,9],        
                            [1,3]]       
                            
    
       
    ################################
    # classical Song Probabilities
    ################################
    
    # self-derived data based on my experiences with classical music
    
    probabilityTypeclassical = [["unison",30],
                                ["octave",5],
                                ["step",45],
                                ["skip",20]]
    
    probabilityIntervalclassical = [[4,33],  # perfect fifth
                                    [3,12],  # perfect fourth
                                    [2,50],  # third
                                    [5,5]]   # sixth
                                    
    probabilityHarmonyclassical = [[4,33],  # perfect fifth
                                   [3,17],  # perfect fourth
                                   [2,50]]  # third
                               
    probabilityLengthclassical = [[16,15], 
                                  [8,40],       
                                  [4,20],      
                                  [3,8], 
                                  [2,10],        
                                  [1,3]]  
    
    #####################################
    # country Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv country
    
    probabilityTypecountry = [["unison",20],
                           ["octave",8],
                           ["step",46],
                           ["skip",26]]
    
    probabilityIntervalcountry = [[5,15],  # perfect fifth
                               [4,15],  # perfect fourth
                               [3,45], # third
                               [7,25]] # minor 7th
    
    probabilityHarmonycountry = [[5,15],  # perfect fifth
                              [4,20],  # perfect fourth
                              [3,65]]  # third
                               
    probabilityLengthcountry = [[12,30],
                             [6,40],
                             [8,25],
                             [4,5]]
                             
    
    #####################################
    # disco Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv disco
    
    probabilityTypedisco = [["unison",30],
                           ["octave",5],
                           ["step",50],
                           ["skip",15]]
    
    probabilityIntervaldisco = [[5,45],  # perfect fifth
                               [4,45],  # perfect fourth
                               [3,5], # third
                               [7,5]] # minor 7th
    
    probabilityHarmonydisco = [[5,50],  # perfect fifth
                              [4,40],  # perfect fourth
                              [3,10]]  # third
                               
    probabilityLengthdisco = [[12,10],
                             [6,10],
                             [8,25],
                             [4,55]]
                             
    
    
    #####################################
    # hiphop Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv hiphop
    
    probabilityTypehiphop = [["unison",30],
                           ["octave",5],
                           ["step",30],
                           ["skip",35]]
    
    probabilityIntervalhiphop = [[5,35],  # perfect fifth
                               [4,15],  # perfect fourth
                               [3,35], # third
                               [7,15]] # minor 7th
    
    probabilityHarmonyhiphop = [[5,20],  # perfect fifth
                              [4,20],  # perfect fourth
                              [3,60]]  # third
                               
    probabilityLengthhiphop = [[12,30],
                             [6,15],
                             [8,40],
                             [4,15]]
                             
    #####################################
    # jazz Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv jazz
    
    probabilityTypejazz = [["unison",20],
                           ["octave",2],
                           ["step",60],
                           ["skip",18]]
    
    probabilityIntervaljazz = [[5,5],  # perfect fifth
                               [4,5],  # perfect fourth
                               [3,65], # third
                               [7,25]] # minor 7th
    
    probabilityHarmonyjazz = [[5,30],  # perfect fifth
                              [4,20],  # perfect fourth
                              [3,50]]  # third
                               
    probabilityLengthjazz = [[12,40],
                             [6,40],
                             [8,15],
                             [4,5]]
    
    #####################################
    # metal Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv metal
    
    probabilityTypemetal = [["unison",20],
                           ["octave",45],
                           ["step",30],
                           ["skip",5]]
    
    probabilityIntervalmetal = [[5,15],  # perfect fifth
                               [4,15],  # perfect fourth
                               [3,25], # third
                               [7,45]] # minor 7th
    
    probabilityHarmonymetal = [[5,33],  # perfect fifth
                              [4,33],  # perfect fourth
                              [3,34]]  # third
                               
    probabilityLengthmetal = [[12,25],
                             [6,25],
                             [8,25],
                             [4,25]]
                             
    #####################################
    # pop Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with improv pop
    
    probabilityTypepop = [["unison",32],
                           ["octave",5],
                           ["step",28],
                           ["skip",35]]
    
    probabilityIntervalpop = [[5,32],  # perfect fifth
                               [4,18],  # perfect fourth
                               [3,35], # third
                               [7,15]] # minor 7th
    
    probabilityHarmonypop = [[5,25],  # perfect fifth
                              [4,20],  # perfect fourth
                              [3,55]]  # third
                               
    probabilityLengthpop = [[12,27],
                             [6,15],
                             [8,43],
                             [4,15]]
    
    #####################################
    # reggae Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with reggae
    
    probabilityTypereggae = [["unison",27],
                           ["octave",15],
                           ["step",28],
                           ["skip",30]]
    
    probabilityIntervalreggae = [[5,32],  # perfect fifth
                               [4,18],  # perfect fourth
                               [3,35], # third
                               [7,15]] # minor 7th
    
    probabilityHarmonyreggae = [[5,25],  # perfect fifth
                              [4,20],  # perfect fourth
                              [3,55]]  # third
                               
    probabilityLengthreggae = [[12,25],
                             [6,15],
                             [8,43],
                             [4,17]]
    
    #####################################
    # rock Song Probabilities
    #####################################
    
    # self-derived data based on my experiences with rock
    
    probabilityTyperock = [["unison",23],
                           ["octave",17],
                           ["step",30],
                           ["skip",30]]
    
    probabilityIntervalrock = [[5,30],  # perfect fifth
                               [4,18],  # perfect fourth
                               [3,35], # third
                               [7,17]] # minor 7th
    
    probabilityHarmonyrock = [[5,29],  # perfect fifth
                              [4,24],  # perfect fourth
                              [3,47]]  # third
                               
    probabilityLengthrock = [[12,25],
                             [6,15],
                             [8,43],
                             [4,17]]
    
    ################################
    # Replacement Probabilities
    ################################
    
    restProbability = [[1,5], # True
                       [0,95]] # False
    
    harmonyProbabilityblue = [[1,40],
                              [0,60]]
    
    harmonyProbabilityclassical = [[1,50], # True
                                   [0,50]] # False
      
    harmonyProbabilitycountry = [[1,45],
                              [0,55]]    
    
    harmonyProbabilitydisco = [[1,50], # True
                               [0,50]] # False
    
    harmonyProbabilityhiphop = [[1,50],
                                [0,50]]
    
    harmonyProbabilityjazz = [[1,35],
                              [0,65]]
    
    harmonyProbabilitymetal = [[1,50], # True
                               [0,50]] # False
    
    harmonyProbabilitypop = [[1,40],
                             [0,60]]
    
    harmonyProbabilityreggae = [[1,35],
                              [0,65]]
    
    harmonyProbabilityrock = [[1,35],
                              [0,65]]
    
    ##############################
    # Randomly Assigned Variables
    ##############################
                     
    meters = ["2/4","3/4","4/4","5/4"] # list of choosable meters
    meter = random.choice(meters) # selects a random meter to compose upon
    
    
    #####################
    # Scale Creators
    #####################

    def makejazzScale(noteList): # create jazz scale based on random
        list = [0,2,3,4,5,7,9,10,11]
        newList = []
        for note in list:
            newList.append(noteList[note])
        return newList
    
    def makeMajMinScale(noteList,mood): # takes current noteList and only leave
                                         # in necessary notes
        majorList = [0,2,4,5,7,9,11]
        minorList = [0,2,3,5,7,8,11]
        newList = []
        if (mood == "Major"):
            for note in majorList:
                newList.append(noteList[note])
        if (mood == "Minor"):
            for note in minorList:
                newList.append(noteList[note])
        return newList
    
    def scaleTransformer(noteList,key): # takes current noteList and starts it
                                        # from the given key's tonic note
        simplifiedList = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
        newList = noteList
        if (not key in simplifiedList):
            if (key == "Db"):
                key = "C"
            if (key == "Eb"):
                key = "D#"
            if (key == "Gb"):
                key = "F#"
            if (key == "Ab"):
                key = "G#"
            if (key == "Bb"):
                key = "A#"
        while (newList[0] != key):
            newList.append(newList.pop(0))
        return newList
        
    
    ###################
    # Genre Dispatcher
    ###################
    
    if (genre == "blue"):
        probabilityType = probabilityTypeblue
        probabilityInterval = probabilityIntervalblue
        probabilityLength = probabilityLengthblue
        probabilityHarmony = probabilityHarmonyblue
        harmonyProbability = harmonyProbabilityblue
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = blueDominant
    if (genre == "classical"):
        probabilityType = probabilityTypeclassical
        probabilityInterval = probabilityIntervalclassical
        probabilityLength = probabilityLengthclassical
        probabilityHarmony = probabilityHarmonyclassical
        harmonyProbability = harmonyProbabilityclassical
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = classicalDominant
    if (genre == "country"):
        probabilityType = probabilityTypecountry
        probabilityInterval = probabilityIntervalcountry
        probabilityLength = probabilityLengthcountry
        probabilityHarmony = probabilityHarmonycountry
        harmonyProbability = harmonyProbabilitycountry
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = countryDominant
    if (genre == "disco"):
        probabilityType = probabilityTypedisco
        probabilityInterval = probabilityIntervaldisco
        probabilityLength = probabilityLengthdisco
        probabilityHarmony = probabilityHarmonydisco
        harmonyProbability = harmonyProbabilitydisco
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = discoDominant
    if (genre == "hiphop"):
        probabilityType = probabilityTypehiphop
        probabilityInterval = probabilityIntervalhiphop
        probabilityLength = probabilityLengthhiphop
        probabilityHarmony = probabilityHarmonyhiphop
        harmonyProbability = harmonyProbabilityhiphop
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = hiphopDominant
    if (genre == "jazz"):
        probabilityType = probabilityTypejazz
        probabilityInterval = probabilityIntervaljazz
        probabilityLength = probabilityLengthjazz
        probabilityHarmony = probabilityHarmonyjazz
        harmonyProbability = harmonyProbabilityjazz
        noteList = scaleTransformer(noteList,key)
        scale = makejazzScale(noteList)
        Dominant = jazzDominant
    if (genre == "metal"):
        probabilityType = probabilityTypemetal
        probabilityInterval = probabilityIntervalmetal
        probabilityLength = probabilityLengthmetal
        probabilityHarmony = probabilityHarmonymetal
        harmonyProbability = harmonyProbabilitymetal
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = metalDominant
    if (genre == "pop"):
        probabilityType = probabilityTypepop
        probabilityInterval = probabilityIntervalpop
        probabilityLength = probabilityLengthpop
        probabilityHarmony = probabilityHarmonypop
        harmonyProbability = harmonyProbabilitypop
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = popDominant
    if (genre == "reggae"):
        probabilityType = probabilityTypereggae
        probabilityInterval = probabilityIntervalreggae
        probabilityLength = probabilityLengthreggae
        probabilityHarmony = probabilityHarmonyreggae
        harmonyProbability = harmonyProbabilityreggae
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = reggaeDominant
    if (genre == "rock"):
        probabilityType = probabilityTyperock
        probabilityInterval = probabilityIntervalrock
        probabilityLength = probabilityLengthrock
        probabilityHarmony = probabilityHarmonyrock
        harmonyProbability = harmonyProbabilityrock
        noteList = scaleTransformer(noteList,key)
        scale = makeMajMinScale(noteList,mood)
        Dominant = rockDominant
    
    #######################
    # Calculated Variables
    #######################
        
    def getMainMeasures(totalLength,tempo,meter):
        return int((tempo*MinsPerSec*totalLength/int(meter[0])))
        
    def getEndingMeasures(totalLength):
        return totalLength//8
    
    def getProbability(probabilityTable):
        probabilities = expandProbability(probabilityTable)
        return random.choice(probabilities)
    
    mainLength = getMainMeasures(totalLength,tempo,meter)
    endLength = getEndingMeasures(mainLength)
    mainLength = mainLength - endLength
    
    def melodyMainGenerator(mainLength):
        global finalPieceMelody
        skipOctave = False # checks if an octave is ever done
        octaveDirection = None # checks which direction it went in
        startingNotes = [noteList[0],noteList[Dominant]]
        # piece can either start on tonic or dominant
        startingNote = random.choice(startingNotes)
        startingLength = getProbability(probabilityLength)
        startingNote = ("%s%i" %(startingNote.lower(),MSO),int(startingLength))
        finalPieceMelody.append(startingNote)
        currNote = startingNote
        totalMeasures = 1/int(startingLength)
        
        ##### appendNotes #####
        def appendOctave(length):
            nonlocal skipOctave
            nonlocal octaveDirection
            nonlocal currNote
            nonlocal totalMeasures
            nonlocal mainLength
            if (skipOctave == False):
                directions = ["down","up"]
                direction = random.choice(directions)
                if (direction == "up"):
                    newNote = ("%s%i" %(currNote[0][:-1],
                        int(currNote[0][-1:])+1),length)
                    finalPieceMelody.append(newNote)
                    octaveDirection = "up"
                    currNote = newNote
                    totalMeasures += (1/length)
                elif (direction == "down"):
                    newNote = ("%s%i" %(currNote[0][:-1],
                        int(currNote[0][-1:])-1),length)
                    finalPieceMelody.append(newNote)
                    octaveDirection = "down"
                    currNote = newNote
                    totalMeasures += (1/length)
                skipOctave = True
            if (skipOctave == True):
                if (octaveDirection == "up"):
                    newNote = ("%s%i" %(currNote[0][:-1],
                            int(currNote[0][-1:])-1),length)
                    finalPieceMelody.append(newNote)
                    octaveDirection = None
                    currNote = newNote
                    totalMeasures += (1/length)
                if (octaveDirection == "down"):
                    newNote = ("%s%i" %(currNote[0][:-1],
                            int(currNote[0][-1:])+1),length)
                    finalPieceMelody.append(newNote)
                    octaveDirection = None
                    currNote = newNote
                    totalMeasures += (1/length)
                skipOctave = False
        
        def appendSkip(length):
            nonlocal skipOctave
            nonlocal octaveDirection
            nonlocal currNote
            nonlocal totalMeasures
            nonlocal mainLength
            if (skipOctave == False):
                directions = ["down","up"]
                direction = random.choice(directions)
                stepSize = int(getProbability(probabilityInterval))
                if (direction == "up"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) + stepSize) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        totalMeasures += (1/length)
                    else:
                        return False
                elif (direction == "down"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) - stepSize) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        totalMeasures += (1/length)
                    else:
                        return False
            if (skipOctave == True):
                if (octaveDirection == "up"):
                    stepSize = int(getProbability(probabilityInterval))
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) - stepSize) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        skipOctave = False
                        totalMeasures += (1/length)
                    else:
                        return False
                if (octaveDirection == "down"):
                    stepSize = int(getProbability(probabilityInterval))
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) + stepSize) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        skipOctave = False
                        totalMeasures += (1/length)
                    else:
                        return False
                        
        def appendStep(length):
            nonlocal skipOctave
            nonlocal octaveDirection
            nonlocal currNote
            nonlocal totalMeasures
            nonlocal mainLength
            if (skipOctave == False):
                directions = ["down","up"]
                direction = random.choice(directions)
                if (direction == "up"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) + step) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        totalMeasures += (1/length)
                    else:
                        return False
                elif (direction == "down"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) - step) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        totalMeasures += (1/length)
                    else:
                        return False
            if (skipOctave == True):
                if (octaveDirection == "up"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) - step) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        skipOctave = False
                        totalMeasures += (1/length)
                    else:
                        return False
                if (octaveDirection == "down"):
                    newNote = (("%s%i" %((noteList[(noteList.index(
                        currNote[0][0].upper()) + step) %
                        len(noteList)]).lower(),MSO),length))
                    if (newNote[0][:-1].upper() in scale):
                        finalPieceMelody.append(newNote)
                        currNote = newNote
                        skipOctave = False
                        totalMeasures += (1/length)
                    else:
                        return False
                        
        def appendUnison(length,noteType):
            nonlocal skipOctave
            nonlocal octaveDirection
            nonlocal currNote
            nonlocal totalMeasures
            nonlocal mainLength
            if (noteType == "skip"):
                returnValue = appendSkip(length)
                if (returnValue != False):
                    unisonLocations.append(len(finalPieceMelody))
                    appendSkip(length)
                    totalMeasures += (1/length)
                else:
                    return False
            if (noteType == "step"):
                returnValue = appendSkip(length)
                if (returnValue != False):
                    unisonLocations.append(len(finalPieceMelody))
                    appendStep(length)
                    totalMeasures += (1/length)
                else:
                    return False
                    
    
        while (totalMeasures < mainLength):
            if ((mainLength - totalMeasures) < 1):
                length = 1/(mainLength - totalMeasures)
            else:
                length = int(getProbability(probabilityLength))
            type = getProbability(probabilityType)
            if (type == "octave"):
                appendOctave(length)
            if (type == "skip"):
                returnValue = appendSkip(length)
                if (returnValue != False):
                    appendSkip(length)
            if (type == "step"):
                returnValue = appendStep(length)
                if (returnValue != False):
                    appendStep(length)
            if (type == "unison"):
                noteTypes = ["step","step","skip"]
                noteType = random.choice(noteTypes)
                returnValue = appendUnison(length,noteType)
                if (returnValue != False):
                    appendUnison(length,noteType)
    
    def melodyEndGenerator():
        global finalPieceMelody
        skipOctave = False # checks if an octave is ever done
        octaveDirection = None # checks which direction it went in
        endingNotes = ["%s%d" %(noteList[-1].lower(),3),
            "%s%d" %(noteList[0].lower(),4)] # subtonic, tonic
        finalPieceMelody[-2] = (endingNotes[0],finalPieceMelody[-2][1])
        finalPieceMelody[-1] = (endingNotes[-1],finalPieceMelody[-1][1])
        
        
    def harmonyMainGenerator(mainLength):
        global finalPieceHarmony
        extensions = {}
        auxList = []
        
        def appendHarmony(note,length):
            interval = getProbability(probabilityHarmony)
            newNote = (("%s%i" %((noteList[(noteList.index(
                        note.upper()) - int(interval)) %
                        len(noteList)]).lower(),HSO),length))
            if (not newNote[0][:-1].upper() in scale):
                newNote = (("%s%i" %((noteList[(noteList.index(
                        note.upper()) - int(interval) - 1) %
                        len(noteList)]).lower(),HSO),length))
            finalPieceHarmony.append(newNote)
        
        def appendHarmonyUnison(location,noteRef,lengthRef):
            unison = ("%s%i" %(noteRef,HSO),lengthRef)
            finalPieceHarmony[location] = unison
                
        for note in range(len(finalPieceMelody)):
            noteReference = finalPieceMelody[note][0][:-1]
            lengthReference = finalPieceMelody[note][1]
            appendHarmony(noteReference,lengthReference)
        for note in unisonLocations:
            noteReference = finalPieceMelody[note][0][:-1]
            lengthReference = finalPieceMelody[note][1]
            appendHarmonyUnison(note,noteReference,lengthReference)
        
        for note in range(len(finalPieceHarmony)):
            if (not note in unisonLocations and note != 0 and finalPieceHarmony[note-1] != "r"):
                remove = getProbability(harmonyProbability)
                if (int(remove) == 1):
                    extensions[note] = finalPieceHarmony[note][1]
                    finalPieceHarmony[note] = "r"
        
        for note in range(len(finalPieceHarmony)):
            if (finalPieceHarmony[note] == "r"):
                newLength = 1/(1/extensions[note] + 1/finalPieceHarmony[note-1][1])
                newNote = (finalPieceHarmony[note-1][0],newLength)
                finalPieceHarmony[note-1] = newNote
        
        for note in range(len(finalPieceHarmony)):
            if (finalPieceHarmony[note] != "r"):
                auxList.append(finalPieceHarmony[note])
        finalPieceHarmony = auxList


    def melodyModifications():
        for note in range(len(finalPieceMelody)):
            restProb = getProbability(restProbability)
            if (int(restProb) == 1):
                rest = ("r",finalPieceMelody[note][1])
                finalPieceMelody[note] = rest
                
    def lessRandom():
        options = [1,2]
        option = random.choice(options)
        if (option == 1):
            positions = [note for note in range(len(finalPieceMelody))]
            positions = positions[:-20]
            position = random.choice(positions)
            replaceSeg = finalPieceMelody[position:position+16]
            replaceSegNums = positions[position:position+16]
            for i in replaceSegNums:
                positions.remove(i)
            replace = random.choice(positions)
            for note in range(replace,replace+16):
                finalPieceMelody[note] = replaceSeg[0]
                replaceSeg.pop(0)
        if (option == 2):
            positions = [note for note in range(len(finalPieceMelody))]
            positions = positions[:-20]
            position = random.choice(positions)
            replaceSeg = finalPieceMelody[position:position+16]
            replaceSegNums = positions[position:position+16]
            for i in replaceSegNums:
                positions.remove(i)
            replace = random.choice(positions)
            for note in range(replace,replace+16):
                finalPieceMelody[note] = replaceSeg[-1]
                replaceSeg.pop()
        
        
    melodyMainGenerator(totalLength)
    lessRandom()
    harmonyMainGenerator(totalLength)
    melodyModifications()
    melodyEndGenerator()
    

def playMusic(fileName): # plays wav file at specified location.
                         # code adapted from online forum
# Link: http://stackoverflow.com/questions/17657103/how-to-play-wav-file-in-python
# Yes, yes it's StackOverflow but it's just a simple, self-contained function
    chunk = 1024
    #wf = wave.open('%s' %fileName, 'rb')
    wf = wave.open('%s' %fileName)
    p = pyaudio.PyAudio()
    
    stream = p.open(
        format = p.get_format_from_width(wf.getsampwidth()),
        channels = wf.getnchannels(),
        rate = wf.getframerate(),
        output = True)
    data = wf.readframes(chunk)

    #while data != b'':
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(chunk)

    stream.close()
    p.terminate()
    wf.close()

# combine melody and harmony together to make the final piece
#################################################################
##### Borrowed from: http://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python #####
def combine():
    sound1 = AudioSegment.from_file("melody.wav")
    sound2 = AudioSegment.from_file("harmony.wav")
       
    combined = sound1.overlay(sound2)
       
    combined.export("final.wav", format='wav')
#################################################################


# entire GUI was built with reference to this documentation and forum post:
# https://docs.python.org/3/library/tk.html
# http://stackoverflow.com/questions/7546050/switch-between-two-frames-in-tkinter

################################################################################

class AutoMozartGUI(tk.Tk):
    def __init__(self, *args, **kwargs):
        
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("AutoMozart!")
        container = tk.Frame(self)

        container.pack(side="top", fill="both", expand = True)

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (HelpPage,MainPage):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(HelpPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
    
    
    

class HelpPage(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self,text="AutoMozart! - A Probability-Driven Music Composer Written in Python")
        label.pack(pady=20,padx=30)
        label1 = tk.Label(self,text='''Choose to either customize parameters
        or enter in a song title''')
        label1.pack()
        label2 = tk.Label(self,text='''The 3 parameters are Key, Genre, and Mood. 
        These allow you to choose from the set of all musical keys, the 
        genres of blue, classical, country, disco, hiphop, jazz, metal, pop, reggae, 
        or rock, and play that piece 
        in either a Major or Minor tone.''')
        label2.pack(pady=20)
        label3 = tk.Label(self,text='''If you enter in a title, AutoMozart will use
        a Machine Learning algorithm on the entered title and parse out a 
        Key and a Mood. A Genre will be randomly assigned.''')
        label3.pack(pady=20)
        ready = tk.Label(self,text="Ready to get started?")
        ready.pack(pady=10)
        button = tk.Button(self, text="Aight, It's Lit",
                            command=lambda: controller.show_frame(MainPage))
        button.pack(pady=20)
        

class MainPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        instr = tk.Label(self, text="Customize Parameters  OR  Enter a Title")
        instr.pack(pady=40)
        
        mainFrame = tk.Frame(self)
        leftFrame = tk.Frame(mainFrame)
        middleFrame = tk.Frame(mainFrame)
        rightFrame = tk.Frame(mainFrame)
        
        self.keys = tk.StringVar(leftFrame)
        self.keys.set("C") # default key
        self.keyLabel = tk.Label(leftFrame, text="Key")
        self.keyLabel.pack()
        self.key_button = tk.OptionMenu(leftFrame, self.keys,"C","G","D","A","E",
            "B","F#","C#","F","Bb","Eb","Ab","Db","Gb","Cb")
        self.key_button.pack(padx=10)
        
        self.genreLabel = tk.Label(middleFrame, text="Genre")
        self.genreLabel.pack()
        self.genres = tk.StringVar(middleFrame)
        self.genres.set("jazz") # default genre
        self.genre_button = tk.OptionMenu(middleFrame, self.genres, "blue",
                                          "classical", "country", "disco", 
                                          "hiphop", "jazz", "metal", "pop",
                                          "reggae", "reggae")
        self.genre_button.pack(padx=10)
        
        self.moodLabel = tk.Label(rightFrame, text="Mood")
        self.moodLabel.pack()
        self.moods = tk.StringVar(rightFrame)
        self.moods.set("Major") # default mood
        self.mood_button = tk.OptionMenu(rightFrame, self.moods, "Major", "Minor")
        self.mood_button.pack(padx=10)
        
        leftFrame.pack(side=LEFT)
        middleFrame.pack(side=LEFT)
        rightFrame.pack(side=LEFT)
        mainFrame.pack()
        
        self.space1 = tk.Label(self, text=" ")
        self.space1.pack()
        
        self.titleLabel = tk.Label(self, text="Enter Title")
        self.titleLabel.pack()
        self.title = tk.StringVar(self)
        self.titleEntry = tk.Entry(self, textvariable=self.title)
        self.titleEntry.pack()
        
        self.space2 = tk.Label(self, text=" ")
        self.space2.pack()
        
        self.generate_button = tk.Button(self, text="Generate!", command=self.generateDispatcher)
        self.generate_button.pack(pady=30)

        self.space3 = tk.Label(self, text=" ")
        self.space3.pack()
        
        helpButton = tk.Button(self, text="Instructions",
                            command=lambda: controller.show_frame(HelpPage))
        helpButton.pack(pady=10)
        
    def generateDispatcher(self):
        if (not self.title.get().isdigit() and self.title.get() != ""):
            self.generateWithTitle()
        else:
            self.generateWithVariables()

    def generateWithVariables(self):
        generateMusic(self.keys.get(),self.genres.get(),self.moods.get())
        ps.make_wav(finalPieceMelody, fn="melody.wav")
        ps.make_wav(finalPieceHarmony, fn="harmony.wav")
        combine()
        playMusic("final.wav")
        
    def generateWithTitle(self): # uses data from tonalito.com
                                 # RAW data found in SongClassificationData.xlsx
                                 # how to integrate MonkeyLearn is found:
                                 # https://github.com/monkeylearn/monkeylearn-python
        title = self.title.get()
        if (title.lower() == "blue"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/blue_wav/blues.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "classical"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/classical_wav/classical.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "country"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/country_wav/country.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "hiphop"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/hiphop_wav/hiphop.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "disco"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/disco_wav/disco.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "jazz"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/jazz_wav/jazz.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "metal"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/metal_wav/metal.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "pop"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/pop_wav/pop.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "reggae"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/reggae_wav/reggae.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 
        if (title.lower() == "rock"):
            random_music_no = randint(0, 9)
            tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/rock_wav/rock.0000") + str(random_music_no) + str(".wav")
            playMusic(tmp) 

    

app = AutoMozartGUI()
app.mainloop()        
################################################################################

    


In [6]:
from random import randint
random_music_no = randint(0, 9)
tmp = str("C:/Data Science App/W210/genres_Music_Data_wav/disco_wav/disco.0000") + str(random_music_no) + str(".wav")
print(tmp)

C:/Data Science App/W210/genres_Music_Data_wav/disco_wav/disco.00004.wav


In [11]:
import pysynth as ps
test = (('c', 4), ('e', 4), ('g', 4),
		('c5', -2), ('e6', 8), ('d#6', 2))
ps.make_wav(test, fn = "test.wav")

Piano key frequencies (for equal temperament):
Key number	Scientific name	Frequency (Hz)
         1	             A0	         27.50
         2	            A#0	         29.14
         3	             B0	         30.87
         4	             C1	         32.70
         5	            C#1	         34.65
         6	             D1	         36.71
         7	            D#1	         38.89
         8	             E1	         41.20
         9	             F1	         43.65
        10	            F#1	         46.25
        11	             G1	         49.00
        12	            G#1	         51.91
        13	             A1	         55.00
        14	            A#1	         58.27
        15	             B1	         61.74
        16	             C2	         65.41
        17	            C#2	         69.30
        18	             D2	         73.42
        19	            D#2	         77.78
        20	             E2	         82.41
        21	             F2	         87.31
        22	            F#2	  

In [1]:
pwd

'C:\\Users\\shihyuch'

In [2]:
pip install pysynth


The following command must be run outside of the IPython shell:

    $ pip install pysynth

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/
